# Simple example

In [1]:
# Add BipartitePandas to system path, do not run this
# import sys
# sys.path.append('../../..')

## Import the BipartitePandas Package

Make sure to install it using `pip install bipartitepandas`

In [2]:
import bipartitepandas as bpd

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


## Get your data ready

In this example, we simulate data (we set parameters to make data cleaning interesting)

In [3]:
df = bpd.SimBipartite(bpd.sim_params({'firm_size': 10, 'p_move': 0.05})).simulate()
display(df)

,i,j,y,t,l,k,alpha,psi
0,0,199,-0.685658,0,2,1,0.000000,-0.908458
1,0,199,-1.372244,1,2,1,0.000000,-0.908458
2,0,199,0.041542,2,2,1,0.000000,-0.908458
3,0,199,-0.272140,3,2,1,0.000000,-0.908458
4,0,199,-1.514353,4,2,1,0.000000,-0.908458
...,...,...,...,...,...,...,...,...
49995,9999,814,1.672901,0,1,8,-0.430727,0.908458
49996,9999,814,1.035813,1,1,8,-0.430727,0.908458
49997,9999,814,-0.658498,2,1,8,-0.430727,0.908458
49998,9999,814,-0.491157,3,1,8,-0.430727,0.908458


## Columns

BipartitePandas includes seven pre-defined general columns:

#### Required
- $i$: worker id (any type)
- $j$: firm id (any type)
- $y$: income (float or int)

#### Optional
- $t$: time (int)
- $g$: firm type (any type)
- $w$: weight (float or int)
- $m$: move indicator (int)

## Formats

BipartitePandas includes four formats:

- *Long* - each row gives a single observation
- *Collapsed Long* - like *Long*, but employment spells at the same firm are collapsed into a single observation
- *Event Study* - each row gives two consecutive observations
- *Collapsed Event Study* - like *Event Study*, but employment spells at the same firm are collapsed into a single observation

These formats divide general columns differently:

- *Long* - $i$, $j$, $y$, $t$, $g$, $w$, $m$
- *Collapsed Long* - $i$, $j$, $y$, $t1$, $t2$, $g$, $w$, $m$
- *Event Study* - $i$, $j1$, $j2$, $y1$, $y2$, $t1$, $t2$, $g1$, $g2$, $w1$, $w2$, $m$
- *Collapsed Event Study* - $i$, $j1$, $j2$, $y1$, $y2$, $t11$, $t12$, $t21$, $t22$, $g1$, $g2$, $w1$, $w2$, $m$

## Constructing DataFrames

Our simulated data is in *Long* format. How do we construct a *Long* dataframe?

In [4]:
i = df['i']
j = df['j']
y = df['y']
t = df['t']
bdf = bpd.BipartiteDataFrame(i=i, j=j, y=y, t=t)
display(bdf)

,i,j,y,t
0,0,199,-0.685658,0
1,0,199,-1.372244,1
2,0,199,0.041542,2
3,0,199,-0.272140,3
4,0,199,-1.514353,4
...,...,...,...,...
49995,9999,814,1.672901,0
49996,9999,814,1.035813,1
49997,9999,814,-0.658498,2
49998,9999,814,-0.491157,3


Are we sure this is long? Let's check the datatype:

In [5]:
type(bdf)

bipartitepandas.bipartitelong.BipartiteLong

This method works to construct any format! Just make sure not to mix up columns between formats.

## Now that we have our dataframe, let's check out some statistics

In [6]:
bdf.summary()

format: 'BipartiteLong'
number of workers: 10000
number of firms: 1000
number of observations: 50000
mean wage: -0.02521203795801338
median wage: -0.028331960612761227
min wage: -5.723441234409018
max wage: 6.18630618222527
var(wage): 2.688729896582871
no NaN values: False
no duplicates: False
i-t (worker-year) observations unique (None if t column(s) not included): False
no returns (None if not yet computed): None
contiguous 'i' ids (None if not included): False
contiguous 'j' ids (None if not included): False
contiguous 'g' ids (None if not included): None
connectedness (None if ignoring connectedness): None


## Let's clean our data - and make sure the result is leave-one-observation-out connected

*Hint:* want details on all cleaning parameters? Type `bpd.clean_params().describe_all()`, or search through `bpd.clean_params().keys()` for a particular key, and then type `bpd.clean_params().describe(key)`.

In [7]:
bdf = bdf.clean(bpd.clean_params({'connectedness': 'leave_out_observation'}))
display(bdf)

checking required columns and datatypes
sorting rows
dropping NaN observations
generating 'm' column
keeping highest paying job for i-t (worker-year) duplicates (how='max')
dropping workers who leave a firm then return to it (how=False)
making 'i' ids contiguous
making 'j' ids contiguous
computing largest connected set (how='leave_out_observation')
making 'i' ids contiguous
making 'j' ids contiguous
sorting columns
resetting index


,i,j,y,t,m
0,0,0,-0.685658,0,0
1,0,0,-1.372244,1,0
2,0,0,0.041542,2,0
3,0,0,-0.272140,3,0
4,0,0,-1.514353,4,0
...,...,...,...,...,...
44580,8958,309,1.672901,0,0
44581,8958,309,1.035813,1,0
44582,8958,309,-0.658498,2,0
44583,8958,309,-0.491157,3,0


We can check how the summary statistics changed:

In [8]:
bdf.summary()

format: 'BipartiteLong'
number of workers: 8959
number of firms: 877
number of observations: 44585
mean wage: -0.01300692079153176
median wage: -0.016727182320997125
min wage: -5.723441234409018
max wage: 6.18630618222527
var(wage): 2.6784295083727
no NaN values: True
no duplicates: True
i-t (worker-year) observations unique (None if t column(s) not included): True
no returns (None if not yet computed): True
contiguous 'i' ids (None if not included): True
contiguous 'j' ids (None if not included): True
contiguous 'g' ids (None if not included): None
connectedness (None if ignoring connectedness): 'leave_out_observation'


## Converting formats

### *Collapsed* format

In [9]:
display(bdf.collapse())

,i,j,y,t1,t2,w,m
0,0,0,-0.760570,0,4,5,0
1,1,1,2.257238,0,3,4,1
2,1,2,1.045985,4,4,1,1
3,2,3,1.131926,0,1,2,1
4,2,4,1.581097,2,2,1,2
...,...,...,...,...,...,...,...
10851,8955,14,-1.229985,0,4,5,0
10852,8956,187,2.745629,0,0,1,1
10853,8956,304,2.136334,1,4,4,1
10854,8957,188,-0.230613,0,4,5,0


### *Event Study* format

In [10]:
display(bdf.to_eventstudy())

,i,j1,j2,y1,y2,t1,t2,m
0,0,0,0,-0.685658,-0.685658,0,0,0
1,0,0,0,-1.372244,-1.372244,1,1,0
2,0,0,0,0.041542,0.041542,2,2,0
3,0,0,0,-0.272140,-0.272140,3,3,0
4,0,0,0,-1.514353,-1.514353,4,4,0
...,...,...,...,...,...,...,...,...
42835,8958,309,309,1.672901,1.672901,0,0,0
42836,8958,309,309,1.035813,1.035813,1,1,0
42837,8958,309,309,-0.658498,-0.658498,2,2,0
42838,8958,309,309,-0.491157,-0.491157,3,3,0


### *Collapsed Event Study* format

In [11]:
display(bdf.collapse().to_eventstudy())

,i,j1,j2,y1,y2,t11,t12,t21,t22,w1,w2,m
0,0,0,0,-0.760570,-0.760570,0,4,0,4,5,5,0
1,1,1,2,2.257238,1.045985,0,3,4,4,4,1,1
2,2,3,4,1.131926,1.581097,0,1,2,2,2,1,1
3,2,4,5,1.581097,0.300515,2,2,3,4,1,2,1
4,3,6,6,0.525205,0.525205,0,4,0,4,5,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9106,8954,714,508,0.292972,-0.139976,0,0,1,4,1,4,1
9107,8955,14,14,-1.229985,-1.229985,0,4,0,4,5,5,0
9108,8956,187,304,2.745629,2.136334,0,0,1,4,1,4,1
9109,8957,188,188,-0.230613,-0.230613,0,4,0,4,5,5,0


## Generating firm clusters

Notice the new $g$ column.

In [12]:
display(bdf.cluster())

INFO:bipartitelong:firm groups computed
INFO:bipartitelong:dictionary linking firms to clusters generated
INFO:bipartitelong:sorting columns
INFO:bipartitelong:clusters merged into data


,i,j,y,t,g,m
0,0,0,-0.685658,0,3,0
1,0,0,-1.372244,1,3,0
2,0,0,0.041542,2,3,0
3,0,0,-0.272140,3,3,0
4,0,0,-1.514353,4,3,0
...,...,...,...,...,...,...
44580,8958,309,1.672901,0,2,0
44581,8958,309,1.035813,1,2,0
44582,8958,309,-0.658498,2,2,0
44583,8958,309,-0.491157,3,2,0
